In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
import scipy as sp
import calendar
import seaborn as sns
import os
import gc
from scipy.stats import pearsonr
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime, date, timedelta

In [2]:
#!pip install tabulate

In [3]:
#!pip install stargazer

## Digital Fundametals

In [9]:
factor_dict=pd.read_excel('C:/Users/19084/My Backup Files/Digital Assets/dict_hat.xlsx')
factor_dict.head()

,Factor_hat,Category,FactorNum
0,AdrActCnt_hat,Activity,F1
1,BlkCnt_hat,Blocks,F2
2,BlkSizeByte_hat,Blocks,F3
3,BlkSizeMeanByte_hat,Blocks,F4
4,CapMrktCurUSD_hat,MarketCap,F5


# Pre-Process Data Functions

In [10]:
def factor_read(curr):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    factor_data = wkdir + curr + '.csv'
    dfname = 'factor_data'
    dfname = pd.read_csv(factor_data)
    return dfname

In [11]:
def factor_df(df_in,curr):
    df_out=pd.DataFrame(columns=['Time'])
    df_out['Time']=df_in['Time']   
    for col in df_in.columns:
        for j in range(0,len(factor_dict)):
            if col==factor_dict['Factor_hat'].iloc[j]:
                new_col=curr + '_' + factor_dict['FactorNum'].iloc[j]
                df_out[new_col]=df_in[col]
    return df_out           

## Read and process data

In [19]:
btc_factor_05=factor_read('BTC/BTC_2021_05min07-12')
btc_factor_05=btc_factor_05.drop('Unnamed: 0',1)
btc_factor_30=factor_read('BTC/BTC_2021_30min07-12')
btc_factor_30=btc_factor_30.drop('Unnamed: 0',1)
btc_factor_60=factor_read('BTC/BTC_2021_60min07-12')
btc_factor_60=btc_factor_60.drop('Unnamed: 0',1)
eth_factor_05=factor_read('ETH/ETH_2021_05min07-12')
eth_factor_05=eth_factor_05.drop('Unnamed: 0',1)
eth_factor_30=factor_read('ETH/ETH_2021_30min07-12')
eth_factor_30=eth_factor_30.drop('Unnamed: 0',1)
eth_factor_60=factor_read('ETH/ETH_2021_60min07-12')
eth_factor_60=eth_factor_60.drop('Unnamed: 0',1)
btc_factor_05.head()

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except fo

,Time,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,HashRate_hat
0,7/1/2021 0:00,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
1,7/1/2021 0:05,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
2,7/1/2021 0:10,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
3,7/1/2021 0:15,12828.0,1.0,1352485.0,1.993280e+13,6.25,18745187.54,3430.0,285368951.9
4,7/1/2021 0:20,0.0,0.0,NaN,NaN,0.00,18745187.54,0.0,NaN


In [20]:
btc_factor_df_05=factor_df(btc_factor_05,'BTC')
eth_factor_df_05=factor_df(eth_factor_05,'ETH')
btc_factor_df_30=factor_df(btc_factor_30,'BTC')
eth_factor_df_30=factor_df(eth_factor_30,'ETH')
btc_factor_df_60=factor_df(btc_factor_60,'BTC')
eth_factor_df_60=factor_df(eth_factor_60,'ETH')
btc_factor_df_05.head()

,Time,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,BTC_F20,BTC_F21,BTC_F13
0,7/1/2021 0:00,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
1,7/1/2021 0:05,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
2,7/1/2021 0:10,0.0,0.0,NaN,NaN,0.00,18745181.29,0.0,NaN
3,7/1/2021 0:15,12828.0,1.0,1352485.0,1.993280e+13,6.25,18745187.54,3430.0,285368951.9
4,7/1/2021 0:20,0.0,0.0,NaN,NaN,0.00,18745187.54,0.0,NaN


In [70]:
eth_factor_df_05.head()

,Time,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
0,7/1/2021 0:00,4395,19,57028.42105,6.651530e+15,39.8125,116508313.1,3719,2942,433.795193
1,7/1/2021 0:05,5092,28,47454.57143,6.657880e+15,57.8125,116508370.9,4221,3124,603.231056
2,7/1/2021 0:10,4449,22,56397.04545,6.660370e+15,44.0000,116508414.9,3567,2472,504.937452
3,7/1/2021 0:15,3610,17,53198.23529,6.662040e+15,35.8125,116508450.7,2763,2362,387.609647
4,7/1/2021 0:20,5568,24,55091.95833,6.639580e+15,49.8125,116508500.5,4685,4340,425.376776


## Merge BTC-ETH

In [21]:
btc_eth_factor_df_05 = pd.merge(btc_factor_df_05,eth_factor_df_05, on='Time', how='outer')
btc_eth_factor_df_30 = pd.merge(btc_factor_df_30,eth_factor_df_30, on='Time', how='outer')
btc_eth_factor_df_60 = pd.merge(btc_factor_df_60,eth_factor_df_60, on='Time', how='outer')
btc_eth_factor_clean_df_05=btc_eth_factor_df_05.dropna().copy()
btc_eth_factor_clean_df_30=btc_eth_factor_df_30.dropna().copy()
btc_eth_factor_clean_df_60=btc_eth_factor_df_60.dropna().copy()
btc_eth_factor_clean_df_05.head()

,Time,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,BTC_F20,BTC_F21,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
3,7/1/2021 0:15,12828.0,1.0,1352485.0,1.993280e+13,6.25,18745187.54,3430.0,285368951.9,3610.0,17.0,53198.23529,6.662040e+15,35.8125,116508450.7,2763.0,2362.0,387.609647
6,7/1/2021 0:30,16135.0,1.0,1447337.0,1.993280e+13,6.25,18745193.79,2755.0,285368951.9,3876.0,16.0,56232.87500,6.624590e+15,32.0000,116508581.9,3178.0,2484.0,334.575334
7,7/1/2021 0:35,15300.0,1.0,1367581.0,1.993280e+13,6.25,18745200.04,2355.0,285368951.9,4153.0,19.0,50522.57895,6.608280e+15,39.8125,116508621.7,3383.0,2784.0,410.169339
10,7/1/2021 0:50,15823.0,1.0,1405260.0,1.993280e+13,6.25,18745206.29,1869.0,285368951.9,5563.0,26.0,65728.03846,6.620110e+15,53.8125,116508768.6,4728.0,3957.0,549.843406
12,7/1/2021 1:00,19960.0,2.0,1246867.5,1.993280e+13,12.50,18745218.79,1216.0,570737903.9,4635.0,25.0,48316.44000,6.647140e+15,51.8125,116508868.4,3825.0,3376.0,538.956908


# Prices

In [24]:
def price_returns(curr,t):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    year = '2021'
    price_data = wkdir+ curr + '/' + curr + 'USD_' + year + '_' + t + 'min.csv'
    temp = pd.read_csv(price_data)
    #curr_price = curr+'_USD'
    df_out=pd.DataFrame(columns=['Time',curr +'_USD',curr +'_Ret'])
    df_out['Time']=temp['Date'] #'Date_Time'
    df_out[curr+'_USD']=temp['Close']
    df_out[curr+'_Ret']=temp['pct']
    return df_out
    # return df_out

In [25]:
btc_price_05=price_returns('BTC','5')
eth_price_05=price_returns('ETH','5')
btc_price_30=price_returns('BTC','30')
eth_price_30=price_returns('ETH','30')
btc_price_60=price_returns('BTC','60')
eth_price_60=price_returns('ETH','60')
btc_price_05.head()

,Time,BTC_USD,BTC_Ret
0,2021-01-01 00:00:00,29033.01,0.000000
1,2021-01-01 00:05:00,29004.87,-0.000969
2,2021-01-01 00:10:00,28896.35,-0.003741
3,2021-01-01 00:15:00,28823.07,-0.002536
4,2021-01-01 00:20:00,28867.21,0.001531


## Merge BTC-ETH prices and returns

In [26]:
btc_eth_price_05=pd.merge(btc_price_05,eth_price_05, on='Time', how='outer')
btc_eth_price_30=pd.merge(btc_price_30,eth_price_30, on='Time', how='outer')
btc_eth_price_60=pd.merge(btc_price_60,eth_price_60, on='Time', how='outer')
#btc_eth_price_clean=btc_eth_price.dropna()
btc_eth_price_05['BTC_ETH_Ret']=btc_eth_price_05['BTC_Ret']-btc_eth_price_05['ETH_Ret']
btc_eth_price_30['BTC_ETH_Ret']=btc_eth_price_30['BTC_Ret']-btc_eth_price_30['ETH_Ret']
btc_eth_price_60['BTC_ETH_Ret']=btc_eth_price_60['BTC_Ret']-btc_eth_price_60['ETH_Ret']
btc_eth_price_05.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret
0,2021-01-01 00:00:00,29033.01,0.000000,738.34,0.000000,0.000000
1,2021-01-01 00:05:00,29004.87,-0.000969,737.67,-0.000907,-0.000062
2,2021-01-01 00:10:00,28896.35,-0.003741,734.67,-0.004067,0.000325
3,2021-01-01 00:15:00,28823.07,-0.002536,732.22,-0.003335,0.000799
4,2021-01-01 00:20:00,28867.21,0.001531,731.50,-0.000983,0.002515


In [27]:
btc_eth_price_05['Time']=btc_eth_price_05['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
btc_eth_price_30['Time']=btc_eth_price_30['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
btc_eth_price_60['Time']=btc_eth_price_60['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  

In [28]:
btc_eth_factor_clean_df_05['Time']=btc_eth_factor_clean_df_05['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  
btc_eth_factor_clean_df_30['Time']=btc_eth_factor_clean_df_30['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  
btc_eth_factor_clean_df_60['Time']=btc_eth_factor_clean_df_60['Time'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M"))  

In [29]:
btc_eth_model_05=pd.merge(btc_eth_price_05,btc_eth_factor_clean_df_05, on='Time', how='outer')
btc_eth_model_30=pd.merge(btc_eth_price_30,btc_eth_factor_clean_df_30, on='Time', how='outer')
btc_eth_model_60=pd.merge(btc_eth_price_60,btc_eth_factor_clean_df_60, on='Time', how='outer')
btc_eth_model_05_keep=btc_eth_model_05.dropna()
btc_eth_model_30_keep=btc_eth_model_30.dropna()
btc_eth_model_60_keep=btc_eth_model_60.dropna()
btc_eth_model_05=btc_eth_model_05_keep
btc_eth_model_30=btc_eth_model_30_keep
btc_eth_model_60=btc_eth_model_60_keep

In [30]:
btc_eth_model_05.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1,BTC_F2,BTC_F4,BTC_F6,...,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
52131,2021-07-01 00:15:00,34824.02,-0.000889,2241.68,-0.006171,0.005282,12828.0,1.0,1352485.0,1.993280e+13,...,285368951.9,3610.0,17.0,53198.23529,6.662040e+15,35.8125,116508450.7,2763.0,2362.0,387.609647
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,16135.0,1.0,1447337.0,1.993280e+13,...,285368951.9,3876.0,16.0,56232.87500,6.624590e+15,32.0000,116508581.9,3178.0,2484.0,334.575334
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,15300.0,1.0,1367581.0,1.993280e+13,...,285368951.9,4153.0,19.0,50522.57895,6.608280e+15,39.8125,116508621.7,3383.0,2784.0,410.169339
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,15823.0,1.0,1405260.0,1.993280e+13,...,285368951.9,5563.0,26.0,65728.03846,6.620110e+15,53.8125,116508768.6,4728.0,3957.0,549.843406
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,19960.0,2.0,1246867.5,1.993280e+13,...,570737903.9,4635.0,25.0,48316.44000,6.647140e+15,51.8125,116508868.4,3825.0,3376.0,538.956908


In [38]:
c_btc_eth_model_05=btc_eth_model_05.drop(['BTC_USD','ETH_USD'], axis=1)
c_btc_eth_model_30=btc_eth_model_30.drop(['BTC_USD','ETH_USD'], axis=1)
c_btc_eth_model_60=btc_eth_model_60.drop(['BTC_USD','ETH_USD'], axis=1)

In [39]:
min05_correl = c_btc_eth_model_05.corr()
min30_correl = c_btc_eth_model_30.corr()
min60_correl = c_btc_eth_model_60.corr()

In [71]:
min05_correl.to_csv('05min_correl_matrix_2021.csv',index=False)
min30_correl.to_csv('30min_correl_matrix_2021.csv',index=False)
min60_correl.to_csv('60min_correl_matrix_2021.csv',index=False)

In [41]:
btc_eth_model_05.corr()

,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1,BTC_F2,BTC_F4,BTC_F6,BTC_F14,...,BTC_F13,ETH_F1,ETH_F2,ETH_F4,ETH_F6,ETH_F14,ETH_F20,ETH_F21,ETH_F22,ETH_F13
BTC_USD,1.000000,0.005844,0.912734,0.000789,0.006951,0.103250,-0.000454,0.107259,0.662561,-0.000454,...,0.257634,0.199034,-0.012881,0.138917,0.694785,-0.000346,0.098832,0.149013,0.281885,0.444769
BTC_Ret,0.005844,1.000000,0.003593,0.825481,-0.033332,-0.001665,0.000616,-0.005208,-0.001681,0.000616,...,0.000374,0.008318,0.007387,-0.011695,-0.001972,0.004344,0.003501,0.008507,0.016502,0.005157
ETH_USD,0.912734,0.003593,1.000000,-0.000221,0.005523,0.114898,0.001815,0.113852,0.831103,0.001815,...,0.325417,0.228474,-0.010683,0.143194,0.871788,0.002869,0.101475,0.141104,0.285172,0.561993
ETH_Ret,0.000789,0.825481,-0.000221,1.000000,-0.591631,0.000315,0.001795,-0.000043,-0.009210,0.001795,...,-0.001977,0.003787,0.003329,-0.014364,-0.008744,-0.000394,0.014706,0.005504,0.006418,-0.001803
BTC_ETH_Ret,0.006951,-0.033332,0.005523,-0.591631,1.000000,-0.002937,-0.002299,-0.007362,0.013907,-0.002299,...,0.004035,0.005175,0.004656,0.008730,0.012666,0.006903,-0.021041,0.002406,0.012207,0.010559
BTC_F1,0.103250,-0.001665,0.114898,0.000315,-0.002937,1.000000,0.447890,0.624954,0.130151,0.447890,...,0.469889,0.059475,0.009074,0.024478,0.115198,0.010840,-0.037571,0.041901,0.014154,0.078853
BTC_F2,-0.000454,0.000616,0.001815,0.001795,-0.002299,0.447890,1.000000,-0.222700,-0.017983,1.000000,...,0.906432,-0.005243,-0.005534,0.017561,-0.007990,-0.007108,0.003210,0.000028,0.001591,-0.010403
BTC_F4,0.107259,-0.005208,0.113852,-0.000043,-0.007362,0.624954,-0.222700,1.000000,0.140430,-0.222700,...,-0.144578,0.077822,0.012712,0.001562,0.118208,0.015135,-0.031069,0.048150,0.002399,0.085304
BTC_F6,0.662561,-0.001681,0.831103,-0.009210,0.013907,0.130151,-0.017983,0.140430,1.000000,-0.017983,...,0.373136,0.238143,-0.011261,0.145689,0.958565,0.004247,-0.016640,0.087945,0.241280,0.619877
BTC_F14,-0.000454,0.000616,0.001815,0.001795,-0.002299,0.447890,1.000000,-0.222700,-0.017983,1.000000,...,0.906432,-0.005243,-0.005534,0.017561,-0.007990,-0.007108,0.003210,0.000028,0.001591,-0.010403


## Export Data

In [46]:
btc_eth_model_05.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_05_21.csv',index=False)
btc_eth_model_30.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_30_21.csv',index=False)
btc_eth_model_60.to_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_60_21.csv',index=False)

## Start Here

In [47]:
btc_eth_factor_clean_df_05=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_05_21.csv')
btc_eth_factor_clean_df_30=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_30_21.csv')
btc_eth_factor_clean_df_60=pd.read_csv('C:/Users/19084/My Backup Files/Digital Assets/btc_eth_model_60_21.csv')

# Modeling Returns

## Lag the factors

In [48]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j)
        curr2_col=curr2 + '_F' + str(j)    
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

## Predictive Analysis

In [49]:
def factor_lag(df_in,first_col,last_col):
    df_out=df_in.iloc[:,0:first_col]
    j=first_col
    for j in range(first_col,last_col):
        col_name_old=df_in.columns[j]
        col_name_new='L_'+col_name_old
        df_out[col_name_new]=df_in.iloc[:,j].shift(1)
        j=j+1
    return df_out

In [50]:
shape = btc_eth_model_60.shape
print('Number of Columns : ', shape[1])

Number of Columns :  23


In [51]:
btc_eth_model_05_lags=factor_lag(btc_eth_model_05,6,23)
btc_eth_model_30_lags=factor_lag(btc_eth_model_30,6,23)
btc_eth_model_60_lags=factor_lag(btc_eth_model_60,6,23)
btc_eth_model_lags_05_keep=btc_eth_model_05_lags.dropna()
btc_eth_model_lags_30_keep=btc_eth_model_30_lags.dropna()
btc_eth_model_lags_60_keep=btc_eth_model_60_lags.dropna()
btc_eth_model_05_lags=btc_eth_model_lags_05_keep
btc_eth_model_30_lags=btc_eth_model_lags_30_keep
btc_eth_model_60_lags=btc_eth_model_lags_60_keep

In [55]:
btc_eth_model_05_lags.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_F1,L_BTC_F2,L_BTC_F4,L_BTC_F6,...,L_BTC_F13,L_ETH_F1,L_ETH_F2,L_ETH_F4,L_ETH_F6,L_ETH_F14,L_ETH_F20,L_ETH_F21,L_ETH_F22,L_ETH_F13
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,12828.0,1.0,1352485.0,1.993280e+13,...,285368951.9,3610.0,17.0,53198.23529,6.662040e+15,35.8125,116508450.7,2763.0,2362.0,387.609647
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,16135.0,1.0,1447337.0,1.993280e+13,...,285368951.9,3876.0,16.0,56232.87500,6.624590e+15,32.0000,116508581.9,3178.0,2484.0,334.575334
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,15300.0,1.0,1367581.0,1.993280e+13,...,285368951.9,4153.0,19.0,50522.57895,6.608280e+15,39.8125,116508621.7,3383.0,2784.0,410.169339
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,15823.0,1.0,1405260.0,1.993280e+13,...,285368951.9,5563.0,26.0,65728.03846,6.620110e+15,53.8125,116508768.6,4728.0,3957.0,549.843406
52142,2021-07-01 01:10:00,34835.68,0.000398,2254.65,-0.000545,0.000943,19960.0,2.0,1246867.5,1.993280e+13,...,570737903.9,4635.0,25.0,48316.44000,6.647140e+15,51.8125,116508868.4,3825.0,3376.0,538.956908


## Create model string

In [56]:
lm_string_BTC='BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6+ L_BTC_F14 + L_BTC_F20 + L_BTC_F21 + L_BTC_F13'
lm_string_ETH='ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6+ L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22 + L_ETH_F13'
print(lm_string_BTC)
print(lm_string_ETH)

BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6+ L_BTC_F14 + L_BTC_F20 + L_BTC_F21 + L_BTC_F13
ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6+ L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22 + L_ETH_F13


In [57]:
lag_model_BTC_05=smf.ols(lm_string_BTC, data = btc_eth_model_05_lags).fit().summary()
print(lag_model_BTC_05)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5381
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.780
Time:                        22:43:35   Log-Likelihood:             1.0168e+05
No. Observations:               21431   AIC:                        -2.033e+05
Df Residuals:                   21424   BIC:                        -2.033e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.568e-10   1.03e-10     -1.522      0.1

In [58]:
lag_model_BTC_30=smf.ols(lm_string_BTC, data = btc_eth_model_30_lags).fit().summary()
print(lag_model_BTC_30)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.924
Date:                Wed, 21 Sep 2022   Prob (F-statistic):             0.0731
Time:                        22:43:38   Log-Likelihood:                 32668.
No. Observations:                8424   AIC:                        -6.532e+04
Df Residuals:                    8417   BIC:                        -6.527e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.555e-10   2.26e-10     -1.129      0.2

In [59]:
lag_model_BTC_60=smf.ols(lm_string_BTC, data = btc_eth_model_60_lags).fit().summary()
print(lag_model_BTC_60)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.142
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.335
Time:                        22:43:41   Log-Likelihood:                 15585.
No. Observations:                4401   AIC:                        -3.116e+04
Df Residuals:                    4394   BIC:                        -3.111e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.513e-10   2.62e-10     -0.959      0.3

In [60]:
lag_model_ETH_05=smf.ols(lm_string_ETH, data = btc_eth_model_05_lags).fit().summary()
print(lag_model_ETH_05)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.069
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.379
Time:                        22:43:42   Log-Likelihood:                 97076.
No. Observations:               21431   AIC:                        -1.941e+05
Df Residuals:                   21424   BIC:                        -1.941e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.024e-14   1.53e-14      1.974      0.0

In [61]:
lag_model_ETH_30=smf.ols(lm_string_ETH, data = btc_eth_model_30_lags).fit().summary()
print(lag_model_ETH_30)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.343
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.234
Time:                        22:43:43   Log-Likelihood:                 30925.
No. Observations:                8424   AIC:                        -6.184e+04
Df Residuals:                    8417   BIC:                        -6.179e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.734e-12   5.55e-12      0.673      0.5

In [62]:
lag_model_ETH_60=smf.ols(lm_string_ETH, data = btc_eth_model_60_lags).fit().summary()
print(lag_model_ETH_60)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.122
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.346
Time:                        22:43:47   Log-Likelihood:                 14604.
No. Observations:                4401   AIC:                        -2.919e+04
Df Residuals:                    4394   BIC:                        -2.915e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.286e-13   4.75e-13     -0.271      0.7

# Ratio Models

## Relative factors

In [63]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j)
        curr2_col=curr2 + '_F' + str(j)   
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [64]:
btc_eth_factor_05_ratios=relative_factors(btc_eth_model_05,6,'BTC','ETH')
btc_eth_factor_30_ratios=relative_factors(btc_eth_model_30,6,'BTC','ETH')
btc_eth_factor_60_ratios=relative_factors(btc_eth_model_60,6,'BTC','ETH')
btc_eth_factor_05_ratios.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_ETH_F1_ratio,BTC_ETH_F2_ratio,BTC_ETH_F4_ratio,BTC_ETH_F6_ratio,BTC_ETH_F13_ratio,BTC_ETH_F14_ratio,BTC_ETH_F20_ratio,BTC_ETH_F21_ratio
52131,2021-07-01 00:15:00,34824.02,-0.000889,2241.68,-0.006171,0.005282,3.553463,0.058824,25.423494,0.002992,7.362277e+05,0.174520,0.160891,1.241404
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,4.162797,0.062500,25.738271,0.003009,8.529288e+05,0.195312,0.160891,0.866897
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,3.684084,0.052632,27.068709,0.003016,6.957345e+05,0.156986,0.160891,0.696128
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,2.844329,0.038462,21.379917,0.003011,5.190004e+05,0.116144,0.160891,0.395305
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,4.306365,0.080000,25.806278,0.002999,1.058968e+06,0.241255,0.160891,0.317908


In [65]:
min05_correl_ratio = btc_eth_factor_05_ratios.corr()
min30_correl_ratio = btc_eth_factor_30_ratios.corr()
min60_correl_ratio = btc_eth_factor_60_ratios.corr()
min05_correl_ratio.to_csv('05min_ratio_correl_matrix.csv',index=False)
min30_correl_ratio.to_csv('30min_ratio_correl_matrix.csv',index=False)
min60_correl_ratio.to_csv('60min__ratio_correl_matrix.csv',index=False)

In [66]:
btc_eth_factor_05_ratios_lags=factor_lag(btc_eth_factor_05_ratios,6,14)
btc_eth_factor_30_ratios_lags=factor_lag(btc_eth_factor_30_ratios,6,14)
btc_eth_factor_60_ratios_lags=factor_lag(btc_eth_factor_60_ratios,6,14)
btc_eth_factor_05_ratios_lags_keep=btc_eth_factor_05_ratios_lags.dropna()
btc_eth_factor_30_ratios_lags_keep=btc_eth_factor_30_ratios_lags.dropna()
btc_eth_factor_60_ratios_lags_keep=btc_eth_factor_60_ratios_lags.dropna()
btc_eth_factor_05_ratios_lags=btc_eth_factor_05_ratios_lags_keep
btc_eth_factor_30_ratios_lags=btc_eth_factor_30_ratios_lags_keep
btc_eth_factor_60_ratios_lags=btc_eth_factor_60_ratios_lags_keep
btc_eth_factor_05_ratios_lags.head()

,Time,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_ETH_F1_ratio,L_BTC_ETH_F2_ratio,L_BTC_ETH_F4_ratio,L_BTC_ETH_F6_ratio,L_BTC_ETH_F13_ratio,L_BTC_ETH_F14_ratio,L_BTC_ETH_F20_ratio,L_BTC_ETH_F21_ratio
52134,2021-07-01 00:30:00,34825.91,0.003317,2246.00,0.002679,0.000639,3.553463,0.058824,25.423494,0.002992,7.362277e+05,0.174520,0.160891,1.241404
52135,2021-07-01 00:35:00,34814.48,-0.000328,2252.98,0.003108,-0.003436,4.162797,0.062500,25.738271,0.003009,8.529288e+05,0.195312,0.160891,0.866897
52138,2021-07-01 00:50:00,34846.75,0.000336,2251.78,0.000524,-0.000189,3.684084,0.052632,27.068709,0.003016,6.957345e+05,0.156986,0.160891,0.696128
52140,2021-07-01 01:00:00,34749.05,-0.003086,2249.41,-0.002912,-0.000174,2.844329,0.038462,21.379917,0.003011,5.190004e+05,0.116144,0.160891,0.395305
52142,2021-07-01 01:10:00,34835.68,0.000398,2254.65,-0.000545,0.000943,4.306365,0.080000,25.806278,0.002999,1.058968e+06,0.241255,0.160891,0.317908


In [67]:
lm_string_BTC_ETH_Ratio='BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio)

BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio


In [68]:
ratio_model_05=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_05_ratios_lags).fit().summary()
print(ratio_model_05)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.8965
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.518
Time:                        22:44:03   Log-Likelihood:             1.0168e+05
No. Observations:               21431   AIC:                        -2.033e+05
Df Residuals:                   21422   BIC:                        -2.033e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0001    

In [69]:
ratio_model_30=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_30_ratios_lags).fit().summary()
print(ratio_model_30)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9951
Date:                Wed, 21 Sep 2022   Prob (F-statistic):              0.437
Time:                        22:44:10   Log-Likelihood:                 32666.
No. Observations:                8424   AIC:                        -6.531e+04
Df Residuals:                    8415   BIC:                        -6.525e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0286    

In [152]:
ratio_model_60=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_60_ratios_lags).fit().summary()
print(ratio_model_60)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.284
Date:                Tue, 15 Mar 2022   Prob (F-statistic):              0.247
Time:                        00:12:20   Log-Likelihood:                 11791.
No. Observations:                3861   AIC:                        -2.356e+04
Df Residuals:                    3852   BIC:                        -2.351e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.5244    